## error_correction package tutorial
Authors: Paul Dieterle and Gloria Ha

Description: With this package we can infer parameters for different models of error generation in spindle assembly.

Last updated: 4/14/2020

In [1]:
from error_correction import data_io


For this tutorial we will focus on the most basic, "independent model."  We can denote $N$ as the number of chromosomes in each cell.  In this model, each chromosome has an equal low probability of missegregation in a single division, $p$.  In this case, the probability of $M$ missegregations is given by:
\begin{equation*}
    P(M) = {N\choose M}p^M(1-p)^{N-M}
\end{equation*}

The experimental quantity being measured is the number of chromatids in each daughter cell after a single division.

There is noise inherent to the biological process and measurement noise in counting chromatids.  We have not incorporated measurement noise in our model yet, as we are still trying to think of the functional form of the noise.  When generating synthetic data, we added Poisson noise to the chromatid counts.

**Note to Paul: feel free to change/delete whatever -- the below cells are just to make sure the object loading is working**

In [2]:
data_dir = 'error_correction/example_data/'
high_ind_data = data_io.SyntheticData(data_dir+'params_high_ind.yml', data_dir+'data_high_ind.txt')

In [3]:
high_ind_data.data

,errors,dNk,dNk_w_noise
0,4,0,1
1,2,4,5
2,1,2,2
3,0,0,2
4,1,2,3
...,...,...,...
9995,4,0,1
9996,0,0,2
9997,2,4,7
9998,2,4,5


In [4]:
high_ind_data.params

{'n_cells': 10000, 'n_chrom': 92, 'p_misseg': 0.01}